In [2]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [3]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

from federated_utils_fedavg_copy import *



In [4]:
#declear path to your data
drebin_data_path = r'data\drebin.csv'
malgenome_data_path = r'data\malgenome.csv'
kronodroid_data_path = r'data\kronodroid.csv'
TUANDROMD_data_path=r'data\TUANDROMD.csv'



Drebin_data = pd.read_csv(drebin_data_path, header = None)

Malgenome_data = pd.read_csv(malgenome_data_path)

Tuandromd_data=pd.read_csv(TUANDROMD_data_path)

kronodroid_data=pd.read_csv(kronodroid_data_path)
Kronodroid_data = kronodroid_data.iloc[:,range(1,kronodroid_data.shape[1])]

## non iid FedProx FGSM


In [5]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import random



def fgsm_attack(model, X, y, epsilon,loss, optimizer):
    # Generate adversarial examples
    model.eval()
    X_adv = X.detach().clone()
    X_adv.requires_grad = True
    outputs = model(X_adv)
    loss = loss(outputs, y)
    # print("adver" , loss)
    loss.backward()
    grad_sign = X_adv.grad.data.sign()
    X_adv = X_adv + epsilon * grad_sign
    X_adv = torch.clamp(X_adv, 0, 1)
    train_loader = DataLoader(TensorDataset(torch.tensor(X_adv, dtype=torch.float32),
                                        torch.tensor(y, dtype=torch.float32)),
                          batch_size=32, shuffle=True)
    return train_loader

def train_model_prox(model, train_loader, loss_fn, optimizer, mu=0.01):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Add the proximal term
        for param, param_global in zip(model.parameters(), global_model.parameters()):
            loss += (mu / 2) * torch.norm(param - param_global, p=2)**2
        
        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']

for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients_non_iid(X_train, [tuple(label) for label in y_train.astype(int).tolist()], num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedProx 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedProx-non-iid-FGSM  2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]
 
                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)

                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    # List of training approaches
                    training_approaches = [train_model_prox, fgsm_attack]

                    # Randomly choose between normal training and FGSM attack
                    selected_training_approach = random.choice(training_approaches)

                    # Apply the selected training approach
                    if selected_training_approach == train_model_prox:
                        train_model_prox(local_model, train_loader, loss, optimizer)
                    elif selected_training_approach == fgsm_attack:
                        epsilon = 1
                        X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon, loss, optimizer)
                        train_model_prox(local_model, X_adv, loss, optimizer)
                    # train_model(local_model, train_loader, loss, optimizer)
                    # # FGSM attack
                    # epsilon = 1
                    # X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon,loss, optimizer)                  
                    # train_model(local_model, X_adv, loss, optimizer)

                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()

                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedProx-non-iid-FGSM-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedProx-non-iid-FGSM-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedProx-non-iid-FGSM-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 10
No. of Rounds: 100
---------------------------------------------
Client client_1: {(0,): 924, (1,): 279}
Client client_2: {(0,): 476, (1,): 727}
Client client_3: {(0,): 936, (1,): 267}
Client client_4: {(0,): 497, (1,): 706}
Client client_5: {(0,): 377, (1,): 826}
Client client_6: {(0,): 820, (1,): 383}
Client client_7: {(0,): 462, (1,): 741}
Client client_8: {(0,): 594, (1,): 609}
Client client_9: {(0,): 0, (1,): 1203}
Client client_10: {(0,): 841, (1,): 362}
|=======================|
|Traditional FedProx-non-iid-FGSM  2017|
|=======================|
comm_round: 0 | global_acc: 37.400% | global_loss: 0.7007073163986206 | global_f1: 0.5401709401709401 | global_precision: 0.3703951774949766 | global_recall: 0.9972948602344455 | global_auc: 0.5129243192397308| flobal_FPR: 0.002705139765554554 
comm_round: 1 | global_acc: 38.198% | global_loss: 0.6982423663139343 | global_f1: 0.5431309904153355 | global_

## iid FedProx FGSM


In [5]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import random




def fgsm_attack(model, X, y, epsilon,loss, optimizer):
    # Generate adversarial examples
    model.eval()
    X_adv = X.detach().clone()
    X_adv.requires_grad = True
    outputs = model(X_adv)
    loss = loss(outputs, y)
    # print("adver" , loss)
    loss.backward()
    grad_sign = X_adv.grad.data.sign()
    X_adv = X_adv + epsilon * grad_sign
    X_adv = torch.clamp(X_adv, 0, 1)
    train_loader = DataLoader(TensorDataset(torch.tensor(X_adv, dtype=torch.float32),
                                        torch.tensor(y, dtype=torch.float32)),
                          batch_size=32, shuffle=True)
    return train_loader

def train_model_prox(model, train_loader, loss_fn, optimizer, mu=0.01):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Add the proximal term
        for param, param_global in zip(model.parameters(), global_model.parameters()):
            loss += (mu / 2) * torch.norm(param - param_global, p=2)**2
        
        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']

for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients(X_train, y_train, num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedProx 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedProx-FGSM 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]
 
                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)

                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    # List of training approaches
                    training_approaches = [train_model_prox, fgsm_attack]

                    # Randomly choose between normal training and FGSM attack
                    selected_training_approach = random.choice(training_approaches)

                    # Apply the selected training approach
                    if selected_training_approach == train_model_prox:
                        train_model_prox(local_model, train_loader, loss, optimizer)
                    elif selected_training_approach == fgsm_attack:
                        epsilon = 1
                        X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon, loss, optimizer)
                        train_model_prox(local_model, X_adv, loss, optimizer)
                    # train_model(local_model, train_loader, loss, optimizer)
                    # # FGSM attack
                    # epsilon = 1
                    # X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon,loss, optimizer)                  
                    # train_model(local_model, X_adv, loss, optimizer)

                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()

                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedProx-FGSM-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedProx-FGSM-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedProx-FGSM-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 10
No. of Rounds: 100
---------------------------------------------
|=======================|
|Traditional FedProx-FGSM 2017|
|=======================|
comm_round: 0 | global_acc: 49.435% | global_loss: 0.691255509853363 | global_f1: 0.516989520482693 | global_precision: 0.3990196078431373 | global_recall: 0.7339945897204689 | global_auc: 0.5922399478440317| flobal_FPR: 0.2660054102795311 
comm_round: 1 | global_acc: 63.132% | global_loss: 0.681009829044342 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.6409804220435891| flobal_FPR: 1.0 
comm_round: 2 | global_acc: 63.132% | global_loss: 0.6728878617286682 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.6779525648495175| flobal_FPR: 1.0 
comm_round: 3 | global_acc: 63.132% | global_loss: 0.6653454303741455 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.71474616938

## iid FedProx PGD


In [6]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import random
def pgd_attack(model, X, y, epsilon, alpha, num_iter, loss_fn, optimizer):
    model.eval()
    
    # Create a copy of the input for adversarial perturbation
    X_adv = X.detach().clone()
    X_adv.requires_grad = True

    for _ in range(num_iter):
        outputs = model(X_adv)
        loss = loss_fn(outputs, y)
        optimizer.zero_grad()
        loss.backward()

        # Generate perturbation (gradient ascent on the loss)
        with torch.no_grad():
            grad_sign = X_adv.grad.data.sign()
            X_adv = X_adv + alpha * grad_sign
            # Clip perturbation to be within epsilon
            X_adv = torch.max(torch.min(X_adv, X + epsilon), X - epsilon)
            X_adv = torch.clamp(X_adv, 0, 1)

        X_adv.requires_grad = True  # Enable gradient tracking for the next iteration

    # Create a DataLoader with the adversarial examples
    adv_dataset = TensorDataset(X_adv, y)
    adv_loader = DataLoader(adv_dataset, batch_size=32, shuffle=True)

    return adv_loader

def train_model_prox(model, train_loader, loss_fn, optimizer, mu=0.01):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Add the proximal term
        for param, param_global in zip(model.parameters(), global_model.parameters()):
            loss += (mu / 2) * torch.norm(param - param_global, p=2)**2
        
        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']

for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients(X_train, y_train, num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedProx 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedProx-FGSM 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]
 
                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)

                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                                  batch_size=32, shuffle=True)
    
    
                    # List of training approaches
                    training_approaches = [train_model_prox, pgd_attack]
    
                    # Randomly choose between normal training and PGD attack
                    selected_training_approach = random.choice(training_approaches)
    
                    # Apply the selected training approach
                    if selected_training_approach == train_model_prox:
                        train_model_prox(local_model, train_loader, loss, optimizer)
                    elif selected_training_approach == pgd_attack:
                        epsilon = 0.01
                        alpha = 0.01
                        num_iter = 5
                        X_adv =pgd_attack(local_model,torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32), clients_batched[client].dataset.tensors[1], epsilon, alpha, num_iter, loss, optimizer)
                        train_model_prox(local_model, X_adv, loss, optimizer)

                    # train_model(local_model, train_loader, loss, optimizer)
                    # # FGSM attack
                    # epsilon = 1
                    # X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon,loss, optimizer)                  
                    # train_model(local_model, X_adv, loss, optimizer)

                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()

                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedProx-PGD-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedProx-PGD-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedProx-PGD-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 10
No. of Rounds: 100
---------------------------------------------
|=======================|
|Traditional FedProx-FGSM 2017|
|=======================|
comm_round: 0 | global_acc: 63.132% | global_loss: 0.6734648942947388 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.5928788394632266| flobal_FPR: 1.0 
comm_round: 1 | global_acc: 63.132% | global_loss: 0.6655853986740112 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.6701536236384675| flobal_FPR: 1.0 
comm_round: 2 | global_acc: 63.132% | global_loss: 0.6585378646850586 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.7176175016892286| flobal_FPR: 1.0 
comm_round: 3 | global_acc: 63.132% | global_loss: 0.6522354483604431 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.7443339026615023| flobal_FPR: 1.0 
comm_round: 4 | global_acc: 63.13

## non-iid FedProx PGD


In [6]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import random



def pgd_attack(model, X, y, epsilon, alpha, num_iter, loss_fn, optimizer):
    model.eval()
    
    # Create a copy of the input for adversarial perturbation
    X_adv = X.detach().clone()
    X_adv.requires_grad = True

    for _ in range(num_iter):
        outputs = model(X_adv)
        loss = loss_fn(outputs, y)
        optimizer.zero_grad()
        loss.backward()

        # Generate perturbation (gradient ascent on the loss)
        with torch.no_grad():
            grad_sign = X_adv.grad.data.sign()
            X_adv = X_adv + alpha * grad_sign
            # Clip perturbation to be within epsilon
            X_adv = torch.max(torch.min(X_adv, X + epsilon), X - epsilon)
            X_adv = torch.clamp(X_adv, 0, 1)

        X_adv.requires_grad = True  # Enable gradient tracking for the next iteration

    # Create a DataLoader with the adversarial examples
    adv_dataset = TensorDataset(X_adv, y)
    adv_loader = DataLoader(adv_dataset, batch_size=32, shuffle=True)

    return adv_loader

def train_model_prox(model, train_loader, loss_fn, optimizer, mu=0.01):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Add the proximal term
        for param, param_global in zip(model.parameters(), global_model.parameters()):
            loss += (mu / 2) * torch.norm(param - param_global, p=2)**2
        
        loss.backward()
        optimizer.step()


all_avg = []
all_std = []

n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']

# for d in range(0,1):
for d in range(0, 1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients_non_iid(X_train, [tuple(label) for label in y_train.astype(int).tolist()], num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedProx 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedProx non iid pgd 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]
 
                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)

                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)
                    selected_training_approach = random.choice([train_model_prox, pgd_attack])

                    # List of training approaches    # Apply the selected training approach
                    if selected_training_approach == train_model_prox:
                        train_model_prox(local_model, train_loader, loss, optimizer)
                    elif selected_training_approach == pgd_attack:
                        epsilon = 0.1  # Set your desired value for epsilon
                        alpha = 0.01   # Set your desired value for alpha
                        num_iter = 10   # Set your desired number of iterations
                        X_adv =pgd_attack(local_model,torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32), clients_batched[client].dataset.tensors[1], epsilon, alpha, num_iter, loss, optimizer)
                        train_model_prox(local_model, X_adv, loss, optimizer)
                    # train_model(local_model, train_loader, loss, optimizer)
                    # # FGSM attack
                    # epsilon = 1
                    # X_adv = fgsm_attack(local_model, clients_batched[client].dataset.tensors[0], clients_batched[client].dataset.tensors[1], epsilon,loss, optimizer)                  
                    # train_model(local_model, X_adv, loss, optimizer)

                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()

                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedProx-non-iid-pgd-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedProx-non-iid-pgd-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedProx-non-iid-pgd-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 10
No. of Rounds: 100
---------------------------------------------
Client client_1: {(0,): 763, (1,): 440}
Client client_2: {(0,): 1070, (1,): 133}
Client client_3: {(0,): 1018, (1,): 185}
Client client_4: {(0,): 143, (1,): 1060}
Client client_5: {(0,): 571, (1,): 632}
Client client_6: {(0,): 525, (1,): 678}
Client client_7: {(0,): 140, (1,): 1063}
Client client_8: {(0,): 459, (1,): 744}
Client client_9: {(0,): 832, (1,): 371}
Client client_10: {(0,): 1037, (1,): 166}
|=======================|
|Traditional FedProx non iid pgd 2017|
|=======================|
comm_round: 0 | global_acc: 39.262% | global_loss: 0.69559246301651 | global_f1: 0.5465376023827252 | global_precision: 0.37705479452054796 | global_recall: 0.9927862939585211 | global_auc: 0.7332813862927239| flobal_FPR: 0.007213706041478809 
comm_round: 1 | global_acc: 62.101% | global_loss: 0.6875537037849426 | global_f1: 0.6317829457364341 | glob